<a href="https://colab.research.google.com/github/kozyreviva/Data_Science/blob/main/ML_in_business.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импортируеми библиотеки.

In [ ]:
#установка библиотек
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

Сохраним датафреймы в переменные.

In [ ]:
#сохранение файлов
try:
  df_geo_0 = pd.read_csv("/datasets/geo_data_0.csv")
  df_geo_1 = pd.read_csv("/datasets/geo_data_1.csv")
  df_geo_2 = pd.read_csv("/datasets/geo_data_2.csv")
except:
  df_geo_0 = pd.read_csv("geo_data_0.csv")
  df_geo_1 = pd.read_csv("geo_data_1.csv")
  df_geo_2 = pd.read_csv("geo_data_2.csv")

Откроем каждые файл и посмотрим на содержимое. Также посмотрим информацию о данных.

In [ ]:
#вывод 5 строк таблицы
df_geo_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
#вывод информации
df_geo_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
#вывод 5 строк таблицы
df_geo_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
#вывод информации
df_geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
#вывод 5 строк таблицы
df_geo_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
#вывод информации
df_geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Краткие выводы**

Каждый из датафреймов имеет одинаковое количество объектов похожих по признакам. В датафреймах нет пропусков, поэтому заполнение не требуется. Первый признак "id" в каждой таблице не информативен для построения модели, при разделени таблиц не будем его включать.

Подготовим данные.  
Разделим таблицы на обычные и целевой признаки.

In [ ]:
#разделение признаков
features_geo_0 = df_geo_0.drop(["id","product"],axis =1)
target_geo_0 = df_geo_0["product"]

In [ ]:
#разделение признаков
features_geo_1 = df_geo_1.drop(["id","product"],axis =1)
target_geo_1 = df_geo_1["product"]

In [ ]:
#разделение признаков
features_geo_2 = df_geo_2.drop(["id","product"],axis =1)
target_geo_2 = df_geo_2["product"]

Разделение произведено, пристопим к обучению модели.

## Обучение и проверка модели

Каждый датафрейм будет обучаться на своей собственной модели LinearRegression. Но перед этим разделим данные на обучающую и вылидационную выборки в соотношении 75:25.

In [ ]:
#разделение на обучающую и валидационную выборки
features_geo_0_train, features_geo_0_valid, target_geo_0_train, target_geo_0_valid = train_test_split(features_geo_0,
                                                                                                       target_geo_0,
                                                                                                       test_size = 0.25, 
                                                                                                       random_state = 12345)

In [ ]:
#разделение на обучающую и валидационную выборки
features_geo_1_train, features_geo_1_valid, target_geo_1_train, target_geo_1_valid = train_test_split(features_geo_1,
                                                                                                      target_geo_1,
                                                                                                      test_size = 0.25,
                                                                                                      random_state = 12345)

In [ ]:
#разделение на обучающую и валидационную выборки
features_geo_2_train, features_geo_2_valid, target_geo_2_train, target_geo_2_valid = train_test_split(features_geo_2,
                                                                                                      target_geo_2,
                                                                                                      test_size = 0.25,
                                                                                                      random_state = 12345)

Разделение на выборки произведено, приступим к обучению модели.  
Результы с предсказаниями и правильными ответами сохраним в валидационной выборке.

In [ ]:
#обучение модели и сохранение данных
model = LinearRegression()
model.fit(features_geo_0_train,target_geo_0_train)
predictions = model.predict(features_geo_0_valid)
features_geo_0_valid["product"] = target_geo_0_valid
features_geo_0_valid["predicted"] = predictions

/tmp/ipykernel_37/1917869126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_geo_0_valid["product"] = target_geo_0_valid
/tmp/ipykernel_37/1917869126.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_geo_0_valid["predicted"] = predictions


In [ ]:
#обучение модели и сохранение данных
model = LinearRegression()
model.fit(features_geo_1_train,target_geo_1_train)
predictions = model.predict(features_geo_1_valid)
features_geo_1_valid["product"] = target_geo_1_valid
features_geo_1_valid["predicted"] = predictions

/tmp/ipykernel_37/2015538118.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_geo_1_valid["product"] = target_geo_1_valid
/tmp/ipykernel_37/2015538118.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_geo_1_valid["predicted"] = predictions


In [ ]:
#обучение модели и сохранение данных
model = LinearRegression()
model.fit(features_geo_2_train,target_geo_2_train)
predictions = model.predict(features_geo_2_valid)
features_geo_2_valid["product"] = target_geo_2_valid
features_geo_2_valid["predicted"] = predictions

/tmp/ipykernel_37/1641942003.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_geo_2_valid["product"] = target_geo_2_valid
/tmp/ipykernel_37/1641942003.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_geo_2_valid["predicted"] = predictions


Проверим изменения.

In [ ]:
#вывод таблицы
features_geo_0_valid.head()

,f0,f1,f2,product,predicted
71751,0.948970,-0.057547,2.095727,10.038645,95.894952
80493,0.992974,0.206671,-0.142278,114.551489,77.572583
2655,1.199854,-0.563356,-1.852991,132.603635,77.892640
53233,0.691422,-0.433347,0.564974,169.072125,90.175134
91141,0.420772,0.972638,0.736190,122.325180,70.510088


In [ ]:
#вывод таблицы
features_geo_1_valid.head()

,f0,f1,f2,product,predicted
71751,-0.371866,-1.862494,3.002210,80.859783,82.663314
80493,9.015122,-13.881455,1.995363,53.906522,54.431786
2655,-6.507568,-4.817448,1.003449,30.132364,29.748760
53233,14.560845,-10.667755,1.995175,53.906522,53.552133
91141,6.090476,-4.494723,0.013815,0.000000,1.243856


In [ ]:
#вывод таблицы
features_geo_2_valid.head()

,f0,f1,f2,product,predicted
71751,-1.444717,-3.861599,2.225805,61.212375,93.599633
80493,-1.418617,1.276544,-0.976433,41.850118,75.105159
2655,-4.587649,-0.413199,1.649268,57.776581,90.066809
53233,1.871584,1.619101,4.273555,100.053761,105.162375
91141,-2.028785,4.128167,6.089547,109.897122,115.303310


Выведем средний запас предсказанного сырья для каждого региона.

In [ ]:
#вывод среднего объёма сырья по регионам
print("Предсказанные средний объём сырься в geo_0:",features_geo_0_valid["predicted"].mean())
print("Предсказанные средний объём сырься в geo_1:",features_geo_1_valid["predicted"].mean())
print("Предсказанные средний объём сырься в geo_2:",features_geo_2_valid["predicted"].mean())

Предсказанные средний объём сырься в geo_0: 92.59256778438035
Предсказанные средний объём сырься в geo_1: 68.728546895446
Предсказанные средний объём сырься в geo_2: 94.96504596800489


Выведем квадратный корень из средней квадратичной ошибки 

In [ ]:
#вывод RMSE для регионов
print("RMSE для geo_0:",sqrt(mean_squared_error(features_geo_0_valid["product"],features_geo_0_valid["predicted"])))
print("RMSE для geo_1:",sqrt(mean_squared_error(features_geo_1_valid["product"],features_geo_1_valid["predicted"])))
print("RMSE для geo_2:",sqrt(mean_squared_error(features_geo_2_valid["product"],features_geo_2_valid["predicted"])))

RMSE для geo_0: 37.5794217150813
RMSE для geo_1: 0.893099286775617
RMSE для geo_2: 40.02970873393434


**Выводы**

Обученные модели дали хорошие предсказательные результаты по каждому региону. Из RMSE можно увидеть что регион "geo_1" имеет наименьшую ошибку близкую к 0, остальны регионы имеют почти схожие результаты. При этом регион geo_0 имеет запас предсказанного сырья на ~24 тыс барелей меньше чем другие регионы.

## Подготовка к расчёту прибыли

Подготовим необходимые переменные.  
У нас имеется бюджет на разработку скважин - 10 миллиардов.
Точек исследования - 500.  
Лучших точек для разработки - 200.
Цена за баррель(тысячу) - 450000.

Также cохраним RandomState в переменную.

In [ ]:
#определение переменных
INCOME = 450000
POINTS = 500
BEST_POINTS = 200
BUDGET = 10000000000
state = np.random.RandomState(12345)

Сохраним реальный объём и предсказания в отдельные переменные для всех регионов.

In [ ]:
#сохранение столбцов из датафрейма в отдельные переменные
geo_0_target = features_geo_0_valid["product"]
geo_0_predictions = features_geo_0_valid["predicted"]

In [ ]:
#сохранение столбцов из датафрейма в отдельные переменные
geo_1_target = features_geo_1_valid["product"]
geo_1_predictions = features_geo_1_valid["predicted"]

In [ ]:
#сохранение столбцов из датафрейма в отдельные переменные
geo_2_target = features_geo_2_valid["product"]
geo_2_predictions = features_geo_2_valid["predicted"]

Произведем необходимый объём сырья для безубыточной разработки и сравним со средними в регионах.  

Безубыточный объём получим: бюджет / количество лучших скважин планируемых к разработке / цена за тысячу баррель.

In [ ]:
#нахождение безубыточного объёма
point_non_waste = 10000000000/200/450000
#вывод данных
print("Безубыточный объём сырья:",point_non_waste)
print("Средний объём по geo_0:",geo_0_target.mean())
print("Средний объём по geo_1:",geo_1_target.mean())
print("Средний объём по geo_2:",geo_2_target.mean())

Безубыточный объём сырья: 111.11111111111111
Средний объём по geo_0: 92.07859674082927
Средний объём по geo_1: 68.72313602435997
Средний объём по geo_2: 94.88423280885438


**Выводы** 

Необходимые данные сохранены в отдельные переменные. 

Сравнивая безубыточный объём сырья со средним объёмом в регионах,можно сказать что средняя добыча в каждом регионе как минимум на 20 тысяч баррелей меньше.

Напишем функцию для подсчета прибыли по лучшим предсказанным скважинам.

In [ ]:
#функция нахождения прибыли от разработки
def profit(target, predictions, count):
    probs_sorted = predictions.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return INCOME * selected.sum()

## Расчёт прибыли и рисков 

Перед расчентом прибыли напишем фунцию "bootstrap" в которой применим технику Bootstrap с 1000 выборок, для нахождения распределения прибыли. Фунеция возвращает объектв типа Series.

In [ ]:
#функция техники bootstrap для нахождения распределения прибыли
def bootstrap(target,predictions):
    values = []

    #применение Bootstrap
    for i in range(1000):
        target_subsample = target.sample(n = POINTS, replace = True, random_state = state)
        predictions_subsample = predictions[target_subsample.index]

        values.append(profit(target_subsample,predictions_subsample,BEST_POINTS))

    #перевод в другой формат и возврат значения
    return pd.Series(values)

Применим фунцию для geo_0.

In [ ]:
values_geo_0 = bootstrap(geo_0_target,geo_0_predictions)

Найдем среднее по получившейся прибыли.  
Найдём 95% доверительный интервал.
Посчитаем вероятность убытков.

In [ ]:
#нахождение среднего, доверительного интервала, процент риска убытков
mean_geo_0 = values_geo_0.mean()
upper_geo_0 = values_geo_0.quantile(0.975)
lower_geo_0 = values_geo_0.quantile(0.025)
loss_geo_0 = (values_geo_0 < BUDGET).mean()

Такие же действия произведём с данныеми по другим регионам.

In [ ]:
values_geo_1 = bootstrap(geo_1_target,geo_1_predictions)

In [ ]:
#нахождение среднего, доверительного интервала, процент риска убытков
mean_geo_1 = values_geo_1.mean()
upper_geo_1 = values_geo_1.quantile(0.975)
lower_geo_1 = values_geo_1.quantile(0.025)
loss_geo_1 = (values_geo_1 < BUDGET).mean()

In [ ]:
values_geo_2 = bootstrap(geo_2_target,geo_2_predictions)

In [ ]:
#нахождение среднего и доверительного интервала
mean_geo_2 = values_geo_2.mean()
upper_geo_2 = values_geo_2.quantile(0.975)
lower_geo_2 = values_geo_2.quantile(0.025)
loss_geo_2 = (values_geo_2 < BUDGET).mean()

Все необходимы операции произведены и сохранены в переменные.  

Приступим к поиску региона с наилучшими показателями и минимальным риском убытков.  
Выведем все данные по регионам.

In [ ]:
#вывод данных по региону
print("Средняя прибыль по geo_0:",mean_geo_0)
print("97.5% квантиль:",upper_geo_0)
print("2.5% квантиль:",lower_geo_0)
print("Вероятность убытков по geo_0: {:.1%}".format(loss_geo_0))

Средняя прибыль по geo_0: 10425938526.910593
97.5% квантиль: 10947976353.35837
2.5% квантиль: 9897909905.162064
Вероятность убытков по geo_0: 6.0%


In [ ]:
#вывод данных по региону
print("Средняя прибыль по geo_1:",mean_geo_1)
print("97.5% квантиль:",upper_geo_1)
print("2.5% квантиль:",lower_geo_1)
print("Вероятность убытков по geo_1: {:.1%}".format(loss_geo_1))

Средняя прибыль по geo_1: 10518259493.697325
97.5% квантиль: 10953612982.066908
2.5% квантиль: 10128123231.433084
Вероятность убытков по geo_1: 0.3%


In [ ]:
#вывод данных по региону
print("Средняя прибыль по geo_2:",mean_geo_2)
print("97.5% квантиль:",upper_geo_2)
print("2.5% квантиль:",lower_geo_2)
print("Вероятность убытков по geo_2: {:.1%}".format(loss_geo_2))

Средняя прибыль по geo_2: 10420194005.34405
97.5% квантиль: 10989629939.844574
2.5% квантиль: 9884147390.839989
Вероятность убытков по geo_2: 6.2%


Из выведенных данных видно, что доверительный интервал и средний показатель в регионе geo_1 выше чем у остальных регионов. Также риск убытков составляет 0.3% а в остальных 6 и более процента.

**Общий вывод** 

После произведённой работы можно придти к однозначным выводам.  Регион geo_1 имеет самый низкий процент риска убытков из представленных регионов он ниже порогового значения в 2.5%. Также средний показатель дохода в этом регионе выше на ~95 миллионов чем в остальных регионах. Можно добавить что средний объём сырья хоть и меньше чем в других регионах это никак не повлияет на прибыль.